# Email AI Assistant


## Setup


In [ ]:
!pip install openai
!pip install --upgrade google-api-python-client==1.7.2 \
    google-auth==1.8.0 \
    google-auth-httplib2==0.0.3 \
    google-auth-oauthlib==0.4.1


## OpenAI Functions


In [ ]:
import openai
import getpass
openai.api_key = getpass.getpass("OPENAI_API_KEY")

def summarize_email(email_text):
    prompt = f"Summarize the following email in 2-3 sentences:\n\n{email_text}"
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def extract_action_items(email_text):
    prompt = f"Based on this email, what actions need to be taken? List them clearly:\n\n{email_text}"
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

def draft_reply(email_text):
    prompt = f"Write a professional reply to this email that addresses all concerns:\n\n{email_text}"
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


## Upload Gmail API Credentials


In [ ]:
from google.colab import files
uploaded = files.upload()


## Gmail API Authentication


In [ ]:
import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from bs4 import BeautifulSoup

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def gmail_authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_console()
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return build('gmail', 'v1', credentials=creds)

service = gmail_authenticate()


## Fetch Unread Emails


In [ ]:
def get_unread_emails(service, max_results=5):
    results = service.users().messages().list(userId='me', labelIds=['INBOX', 'UNREAD'], maxResults=max_results).execute()
    messages = results.get('messages', [])
    email_texts = []

    for msg in messages:
        msg_data = service.users().messages().get(userId='me', id=msg['id']).execute()
        payload = msg_data['payload']
        parts = payload.get('parts', [])
        data = ''

        if 'data' in payload.get('body', {}):
            data = payload['body']['data']
        elif parts:
            for part in parts:
                if part.get('mimeType') == 'text/plain':
                    data = part['body']['data']
                    break

        if data:
            decoded = base64.urlsafe_b64decode(data).decode('utf-8')
            email_texts.append(decoded)

    return email_texts


## Run AI Assistant on Emails


In [ ]:
emails = get_unread_emails(service)

for i, email_text in enumerate(emails):
    print("="*50)
    print(f"Email #{i+1}\n")
    summary = summarize_email(email_text)
    actions = extract_action_items(email_text)
    reply = draft_reply(email_text)
    print("Summary:\n", summary)
    print("\nAction Items:\n", actions)
    print("\nDrafted Reply:\n", reply)
    print("="*50 + "\n")
